# RQ4 - Prepare Historical Coocurrence - Middle slice

Uses:

- resources/historical_join.xlsx (HISTORICAL_FILE_JOIN from prepare/1.historical.ipynb)

Generates:

- resources/historical_rulesv5.xlsx


In [1]:
import sys  
sys.path.insert(1, '..')

In [2]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)
df

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111,liferay,liferay-portal,Enterprise Resource Planning,fcd4cd6526c80613eca69927effe931bb6a91444,8209,2024-10-07 12:05:23+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31112,liferay,liferay-portal,Enterprise Resource Planning,cc0ab8103195e543056639c8c2883fcc187c33a2,8210,2024-10-07 17:58:58+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31113,liferay,liferay-portal,Enterprise Resource Planning,f75082029372ab28694650bd3fe9fae4d548351e,8211,2024-10-11 04:36:52+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31114,liferay,liferay-portal,Enterprise Resource Planning,a80a679a86d2867f985bafa510d9d12f1bf051a7,8212,2024-10-11 16:53:22+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0


Grouping the lines to identify which projects have DB clues

In [4]:
df2 = df.groupby(["owner", "name"]).sum()
df2.drop(["part_commit", "isLast"], axis=1, inplace=True)
df2.drop(df2[(df2 == 0).all(axis=1)].index, inplace=True)
df2

,,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,0,0,0,0,0,0,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio,alluxio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
AnySoftKeyboard,AnySoftKeyboard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Athou,commafeed,0,0,0,0,0,0,10,0,0,0,...,0,0,0,0,0,18,0,0,0,0
Atmosphere,atmosphere,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,26,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wildfly,wildfly,0,0,0,0,0,0,203,0,0,0,...,0,0,0,0,0,0,0,0,0,0
xuxueli,xxl-job,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
yacy,yacy_search_server,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


New dataframe filtering only lines that are in the middle slice of the projects

In [5]:
# Função para selecionar o commit da metade
def select_half_commit(group):
    middle_index = len(group) // 2  # Calcula o índice do meio
    return group.iloc[middle_index-1]  # Seleciona o commit do meio

# Ordena os commits dentro de cada projeto (caso não estejam ordenados)
df_sorted = df[~df["isLast"]].sort_values(by=['owner', 'name', 'part_commit'])

# Aplica a função para pegar o commit da metade em cada grupo de projeto
df10 = df_sorted.groupby(['owner', 'name']).apply(select_half_commit)
#df10.drop(["owner", "name", "domain", "sha1", "part_commit", "date_commit", "isLast"], axis=1, inplace=True)
df10

,,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,Activiti,Activiti,Enterprise Resource Planning,d2c8230955ee79c53450f5d1b58682a9e85bbcc9,27,2013-01-29 16:56:43+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio,alluxio,Alluxio,alluxio,Data Management,8d93e3684c6374f80e9645bcf04806056e0497b5,54,2018-04-30 16:39:17+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AlmasB,FXGL,AlmasB,FXGL,Game,9e28ad4ca94e7050ac8eac17b6a95fc4e23dd0de,7,2021-02-06 13:28:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntennaPod,AntennaPod,AntennaPod,AntennaPod,Media,d5db62c70b867f21a74e158e5f7e860c46952062,18,2017-05-31 15:09:20+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AnySoftKeyboard,AnySoftKeyboard,AnySoftKeyboard,AnySoftKeyboard,Other,15bb43e4c828d9c990478ebd3f7b1d6d2950acd0,18,2015-05-07 21:05:07+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xuxueli,xxl-job,xuxueli,xxl-job,Automation,d025f7152749f3a54663206aa188561204de45c3,7,2017-12-22 12:54:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yacy,yacy_search_server,yacy,yacy_search_server,Infrastructure Management,3f746be5d4dc8ffbd65e589c4d0dbed8c1bdf9b4,50,2008-11-06 10:07:53+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zaproxy,zaproxy,zaproxy,zaproxy,Security,d6bf06efe1a09d0ac091457eee014e71fad786e4,18,2014-03-07 22:07:57+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues in the middle slice 

In [6]:
with_dbs = df10.loc[df2.index].copy().astype('bool')
with_dbs

,,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Alluxio,alluxio,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AnySoftKeyboard,AnySoftKeyboard,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Athou,commafeed,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
Atmosphere,atmosphere,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wildfly,wildfly,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
xuxueli,xxl-job,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
yacy,yacy_search_server,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### ASSOCIATION RULES

In [7]:
frq_items = apriori(with_dbs, min_support = 0.021, use_colnames = True)

In [ ]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

In [ ]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending=False)
rules

Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [ ]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]
rules_filter

Saving Results

In [ ]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rulesv5.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)